In [ ]:
import sys
sys.path.insert(0, '../')
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from src.config import PARAM, PATH_SHAPES, PATH_RESOURCES, PATH_RESULTS
from src.geo_data import load_geonames
from src.geoplot import geoplot_points, annotate_geoplot

In [ ]:
loc = 'world'
basemap = geopandas.read_file(PATH_SHAPES / loc)
basemap = basemap[basemap.CNTR_ID != 'AQ']
basemap = basemap.to_crs({'init': 'epsg:3395'}) # mercator

In [ ]:
countries = ['GB', 'NL']
df_geo = pd.read_pickle(PATH_RESOURCES / 'geonames/df_geonames.pkl').query("country_code not in @countries")

In [ ]:
geometry = [Point(xy) for xy in zip(df_geo.longitude, df_geo.latitude)]
crs = {'init' :'epsg:4326'}
gdf_ = geopandas.GeoDataFrame(df_geo, crs=crs, geometry=geometry)
gdf_ = gdf_.to_crs(basemap.crs)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
geoplot_points(ax, basemap, gdf_, 3, title='Distribute plaatsnamen')
plt.savefig(PATH_RESULTS / f"distribution_places_{loc}.png")
plt.tight_layout()

In [ ]:
df = pd.read_pickle(PATH_RESULTS / 'df_counts_totals.pkl').xs('places', level=0)
df = df.rename(columns={'index': 'naam'})

In [ ]:
gdf = gdf_.merge(df, how='inner', left_on='alternate_name', right_index=True)

In [ ]:
fig, axs = plt.subplots(4, 1, sharex=True, sharey=True, figsize=(32, 64))
for ax, batch in zip(axs.reshape(-1), PARAM.lexisnexis.batches):
    ax = geoplot_points(ax, basemap, gdf, gdf[batch].fillna(0), title=batch, factor=12)
    places = gdf.dropna(subset=[batch]).loc[gdf[batch] >= 15]
    annotate_geoplot(ax, places, 'alternate_name')
plt.savefig(PATH_RESULTS / f"places_{loc}.png")
plt.tight_layout()

In [ ]:
n=1e7
limitx = [-0.2 * n, 0.5 * n]
limity = [0.35 * n, 1.15 * n]

fig, axs = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(28, 32))
for ax, batch in zip(axs.reshape(-1), PARAM.lexisnexis.batches):
    ax.set_xlim(limitx)
    ax.set_ylim(limity)
    
    ax = geoplot_points(ax, basemap, gdf, gdf[batch].fillna(0), title=batch, factor=12)
    places = gdf.dropna(subset=[batch]).loc[gdf[batch] >= 5]
    annotate_geoplot(ax, places, 'alternate_name')
plt.savefig(PATH_RESULTS / f"places_europe.png")
plt.tight_layout()